# Initialize Environment

In [1]:
import matplotlib, matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import numpy as np
from scipy.stats import norm
from scipy.io import loadmat
import glob
import pandas as pd
import datetime
import pandas_datareader.tsp as tsp
import pandas_datareader.data as pdr
from pandas_datareader.yahoo.headers import DEFAULT_HEADERS
import requests_cache
#import yfinance as yf
#yf.pdr_override() # https://pypi.org/project/yfinance/

from copy import deepcopy
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Seed Random number generator for repeatability
np.random.seed(19690720)

plt.style.use('default')
#matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams.update({'font.size': 18})

# Load Data
## Data Caches

In [2]:
session = requests_cache.CachedSession(cache_name='cache', 
                                       backend='sqlite', 
                                       expire_after=datetime.timedelta(hours=8))
session.headers = DEFAULT_HEADERS

## TSP
Load TSP data using the pandas_datareader utility. Allow for 8-hour persistent caching. 

In [3]:
tspreader = tsp.TSPReader(start=datetime.date.today() - datetime.timedelta(days=365*25),
                          session=session)
full_tspdata = tspreader.read()

# write the TSP data, last 90 values, in reversed order
full_tspdata.iloc[:400].to_excel('tsp_shareprices.xlsx', sheet_name='TSP Data')

#print(full_tspdata)
tspdata = full_tspdata.drop(['L Income', 'L 2025', 'L 2030', 'L 2035', 'L 2040', 'L 2045', 'L 2050', 'L 2055', 'L 2060', 'L 2065'],axis=1).iloc[::-1]

## Vanguard

In [4]:
try:
    tickers
except NameError:    
    tickers = [ 'VTSAX', 'VSGAX', 'VGSLX', 'VHYAX', 'VTMSX', 'VTCLX', 'VTMFX', 'VBTLX', 'VWAHX', 'VGSTX', 'VIHAX', 'VWIGX' ]

data = pdr.DataReader(
        # tickers list or string as well
        tickers, 'yahoo', session=session
    )
data.head()
#rev_data = data.iloc[::-1]
#rev_data.iloc[:800].to_excel('vg_shareprices.xlsx', sheet_name='ETF Data')

Attributes  Adj Close                                                     \
Symbols         VTSAX      VSGAX      VGSLX VHYAX      VTMSX       VTCLX   
Date                                                                       
2017-01-17  52.109707  46.134720  97.542168   NaN  51.535995  107.367790   
2017-01-18  52.238125  46.299564  97.682724   NaN  51.800564  107.599754   
2017-01-19  52.027157  45.979580  96.707153   NaN  51.280876  107.256424   
2017-01-20  52.201431  46.163811  97.426422   NaN  51.498203  107.618317   
2017-01-23  52.063847  46.115334  98.228401   NaN  51.356461  107.349220   

Attributes                                            ... Volume              \
Symbols         VTMFX     VBTLX     VWAHX      VGSTX  ...  VGSLX VHYAX VTMSX   
Date                                                  ...                      
2017-01-17  25.667650  9.457180  9.362727  18.943472  ...    0.0   NaN   0.0   
2017-01-18  25.685848  9.404246  9.345901  18.927752  ...    0.0   NaN   0.0   
2017-01-19  25.594835  9.386600  9.303840  18.880589  ...    0.0   NaN   0.0   
2017-01-20  25.594835  9.395420  9.278603  18.912033  ...    0.0   NaN   0.0   
2017-01-23  25.567528  9.421891  9.278603  18.927752  ...    0.0   NaN   0.0   

Attributes                                            
Symbols    VTCLX VTMFX VBTLX VWAHX VGSTX VIHAX VWIGX  
Date                                                  
2017-01-17   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2017-01-18   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2017-01-19   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2017-01-20   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2017-01-23   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 72 columns]

# Derived Data

In [5]:
ddata = data['Close']
ddata.head()

Symbols,VTSAX,VSGAX,VGSLX,VHYAX,VTMSX,VTCLX,VTMFX,VBTLX,VWAHX,VGSTX,VIHAX,VWIGX
Date,,,,,,,,,,,,
2017-01-17,56.810001,47.580002,117.980003,NaN,54.540001,115.709999,28.200001,10.72,11.13,24.100000,28.180000,22.25
2017-01-18,56.950001,47.750000,118.150002,NaN,54.820000,115.959999,28.219999,10.66,11.11,24.080000,28.070000,22.24
2017-01-19,56.720001,47.419998,116.970001,NaN,54.270000,115.589996,28.120001,10.64,11.06,24.020000,28.020000,22.27
2017-01-20,56.910000,47.610001,117.839996,NaN,54.500000,115.980003,28.120001,10.65,11.03,24.059999,28.129999,22.32
2017-01-23,56.759998,47.560001,118.809998,NaN,54.349998,115.690002,28.090000,10.68,11.03,24.080000,28.190001,22.43
